# Opacus - Syft Duet - Data Scientist 🥁

## PART 1: Connect to a Remote Duet Server

In [ ]:
import syft as sy
sy.load("opacus")

In [ ]:
duet = sy.join_duet(loopback=True)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 0 : Now STOP and run the Data Owner notebook until Checkpoint 1.

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.conv1 = self.torch_ref.nn.Conv2d(1, 16, 8, 2, padding=3)
        self.conv2 = self.torch_ref.nn.Conv2d(16, 32, 4, 2) 
        self.fc1 = self.torch_ref.nn.Linear(32 * 4 * 4, 32)
        self.fc2 = self.torch_ref.nn.Linear(32, 10)

    def forward(self, x):
        F = self.torch_ref.nn.functional
        # x of shape [B, 1, 28, 28]
        x = F.relu(self.conv1(x))  # -> [B, 16, 14, 14]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 16, 13, 13]
        x = F.relu(self.conv2(x))  # -> [B, 32, 5, 5]
        x = F.max_pool2d(x, 2, 1)  # -> [B, 32, 4, 4]
        x = x.view(-1, 32 * 4 * 4)  # -> [B, 512]
        x = F.relu(self.fc1(x))  # -> [B, 32]
        x = self.fc2(x)  # -> [B, 10]
        return x

In [ ]:
# lets import torch and torchvision just as we normally would
import torch
import torchvision

In [ ]:
# now we can create the model and pass in our local copy of torch
local_model = SyNet(torch)

Next we can get our MNIST Test Set ready using our local copy of torch.

In [ ]:
local_model.modules

In [ ]:
# we need some transforms for the MNIST data set
local_transform_1 = torchvision.transforms.ToTensor()  # this converts PIL images to Tensors
local_transform_2 = torchvision.transforms.Normalize(0.1307, 0.3081)  # this normalizes the dataset

# compose our transforms
local_transforms = torchvision.transforms.Compose([local_transform_1, local_transform_2])

In [ ]:
# Lets define a few settings which are from the original Opacus MNIST example args
# TODO: support secure_rng
batch_size = 64
test_batch_size = 1024
args = {
    "batch_size": batch_size,
    "test_batch_size": test_batch_size,
    "epochs": 10,
    "n_runs": 1,
    "lr": 0.1,
    "sigma": 1.0,
    "max_per_sample_grad_norm": 1.0,
    "delta": 1e-5,
    "no_cuda": False,
    "dry_run": True,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": True,
    "disable_dp": False,
}

In [ ]:
from syft.util import get_root_data_path
# we will configure the test set here locally since we want to know if our Data Owner's
# private training dataset will help us reach new SOTA results for our benchmark test set
test_kwargs = {
    "batch_size": args["test_batch_size"],
}

test_data = torchvision.datasets.MNIST(str(get_root_data_path()), train=False, download=True, transform=local_transforms)
test_loader = torch.utils.data.DataLoader(test_data,**test_kwargs)
test_data_length = len(test_loader.dataset)
print(test_data_length)

Now its time to send the model to our partner’s Duet Server.

Note: You can load normal torch model weights before sending your model.
Try training the model and saving it at the end of the notebook and then coming back and
reloading the weights here, or you can train the same model one using the original script
in `original` dir and load it here as well.

In [ ]:
# service_auth issues?
model = local_model.send(duet)

Lets create an alias for our partner’s torch called `remote_torch` so we can refer to the local torch as `torch` and any operation we want to do remotely as `remote_torch`. Remember, the return values from `remote_torch` are `Pointers`, not the real objects. They mostly act the same when using them with other `Pointers` but you can't mix them with local torch objects.

In [ ]:
remote_torch = duet.torch

In [ ]:
# lets ask to see if our Data Owner has CUDA
has_cuda = False
has_cuda_ptr = remote_torch.cuda.is_available()
has_cuda = bool(has_cuda_ptr.get(
    request_block=True,
    reason="To run test and inference locally",
    timeout_secs=5,  # change to something slower
))
print(has_cuda)

In [ ]:
use_cuda = not args["no_cuda"] and has_cuda
# now we can set the seed
remote_torch.manual_seed(args["seed"])

device = remote_torch.device("cuda" if use_cuda else "cpu")
print(f"Data Owner device is {device.type.get()}")

In [ ]:
# if we have CUDA lets send our model to the GPU
if has_cuda:
    model.cuda(device)
else:
    model.cpu()

Lets get our params, setup an optimizer and a scheduler just the same as the PyTorch MNIST example

In [ ]:
params = model.parameters()

In [ ]:
optimizer = remote_torch.optim.SGD(params, lr=args["lr"], momentum=0)

In [ ]:
remote_opacus = duet.opacus

In [ ]:
# this is the model we give to Opacus because its a real nn.ModulePointer on the other side
# and has all the real methods for searching layers and attaching to them
model.real_module

In [ ]:
sample_size = 60000
noise_multiplier = 1.0
max_grad_norm = 1.0
privacy_engine_ptr = remote_opacus.privacy_engine.PrivacyEngine(
    model.real_module, batch_size=args["batch_size"], sample_size=sample_size,
    noise_multiplier=noise_multiplier, max_grad_norm=max_grad_norm
)

In [ ]:
privacy_engine_ptr.attach(optimizer)

Next we need a training loop so we can improve our remote model. Since we want to train on remote data we should first check if the model is remote since we will be using remote_torch in this function. To check if a model is local or remote simply use the `.is_local` attribute.

In [ ]:
def train(model, torch_ref, train_loader, optimizer, epoch, args, train_data_length, privacy_engine_ptr):
    # + 0.5 lets us math.ceil without the import
    train_batches = round((train_data_length / args["batch_size"]) + 0.5)
    print(f"> Running train in {train_batches} batches")
    
    model.train()
    criterion = torch_ref.nn.CrossEntropyLoss()
    losses = []
    for _batch_idx, data_tuple in enumerate(train_loader):
        data, target = data_tuple[0], data_tuple[1]
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        loss_item_ptr = loss.item()
        local_loss = loss_item_ptr.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5
        )

        if local_loss is not None:
            losses.append(local_loss)

        if not args["disable_dp"]:
            epsilon_tuple = privacy_engine_ptr.get_privacy_spent(args["delta"])
            epsilon_ptr = epsilon_tuple[0].resolve_pointer_type()
            best_alpha_ptr = epsilon_tuple[1].resolve_pointer_type()

            epsilon = epsilon_ptr.get(
                reason="So we dont go over it",
                request_block=True,
                timeout_secs=5
            )
            best_alpha = best_alpha_ptr.get(
                reason="So we dont go over it",
                request_block=True,
                timeout_secs=5
            )

            loss_mean = float("-inf")
            if len(losses) > 0:
                loss_mean = sum(losses) / len(losses)
            if epsilon is None:
                epsilon = float("-inf")
            if best_alpha is None:
                best_alpha = float("-inf")
            print(
                f"Train Epoch: {epoch} \t"
                f"Loss: {loss_mean:.6f} "
                f"(ε = {epsilon:.2f}, δ = {args['delta']}) for α = {best_alpha}"
            )
        else:
            print(f"Train Epoch: {epoch} \t Loss: {loss_mean:.6f}")
            
        if args["dry_run"]:
            break
    
        if _batch_idx >= train_batches - 1:
            print("batch_idx >= train_batches, breaking")
            break

Now we can define a simple test loop very similar to the original PyTorch MNIST example.
This function should expect a remote model from our outer epoch loop, so internally we can call `get` to download the weights to do an evaluation on our machine with our local test set. Remember, if we have trained on private data, our model will require permission to download, so we should use request_block=True and make sure the Data Owner approves our requests. For the rest of this function, we will use local `torch` as we normally would.

In [ ]:
def test_local(model, torch_ref, test_loader, test_data_length):
    # + 0.5 lets us math.ceil without the import
    test_batches = round((test_data_length / args["test_batch_size"]) + 0.5)
    print(f"> Running test_local in {test_batches} batches")

    # download remote model
    if not model.is_local:
        local_model = model.get(
            request_block=True,
            reason="test evaluation",
            timeout_secs=5
        )
    else:
        local_model = model

    local_model.eval()
    criterion = torch_ref.nn.CrossEntropyLoss()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            output = local_model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(
                dim=1, keepdim=True
            )  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            if args["dry_run"]:
                break
                
            if batch_idx >= test_batches - 1:
                print("batch_idx >= test_batches, breaking")
                break

    test_loss /= len(test_loader.dataset)

    print(
        "\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            test_loss,
            correct,
            len(test_loader.dataset),
            100.0 * correct / len(test_loader.dataset),
        )
    )
    return correct / len(test_loader.dataset)

Finally just for demonstration purposes, we will get the built-in MNIST dataset but on the Data Owners side from `remote_torchvision`.

In [ ]:
# we need some transforms for the MNIST data set
remote_torchvision = duet.torchvision

transform_1 = remote_torchvision.transforms.ToTensor()  # this converts PIL images to Tensors
transform_2 = remote_torchvision.transforms.Normalize(0.1307, 0.3081)  # this normalizes the dataset

remote_list = duet.python.List()  # create a remote list to add the transforms to
remote_list.append(transform_1)
remote_list.append(transform_2)

# compose our transforms
transforms = remote_torchvision.transforms.Compose(remote_list)

# The DO has kindly let us initialise a DataLoader for their training set
train_kwargs = {
    "batch_size": args["batch_size"],
}
train_data_ptr = remote_torchvision.datasets.MNIST(str(get_root_data_path()), train=True, download=True, transform=transforms)
train_loader_ptr = remote_torch.utils.data.DataLoader(train_data_ptr,**train_kwargs)

In [ ]:
train_data_length = 60000

## PART 3: Training

In [ ]:
import time

# args["dry_run"] = False  # uncomment to do a full train
print("Starting Training")

for epoch in range(1, args["epochs"] + 1):
    epoch_start = time.time()
    print(f"Epoch: {epoch}")
    # remote training on model with remote_torch
    train(model, remote_torch, train_loader_ptr, optimizer, epoch, args, train_data_length, privacy_engine_ptr)
    # local testing on model with local torch
    test_local(model, torch, test_loader, test_data_length)
    epoch_end = time.time()
    print(f"Epoch time: {int(epoch_end - epoch_start)} seconds")
    if args["dry_run"]:
        break

print("Finished Training")

In [ ]:
local_model = None
if args["save_model"]:
    local_model = model.get(
        request_block=True,
        reason="test evaluation",
        timeout_secs=5
    ).save("./duet_mnist.pt")

## PART 4: Inference

A model would be no fun without the ability to do inference. The following code shows some examples on how we can do this either remotely or locally.

In [ ]:
import matplotlib.pyplot as plt
def draw_image_and_label(image, label):
    fig = plt.figure()
    plt.tight_layout()
    plt.imshow(image, cmap="gray", interpolation="none")
    plt.title("Ground Truth: {}".format(label))
    
def prep_for_inference(image):
    image_batch = image.unsqueeze(0).unsqueeze(0)
    image_batch = image_batch * 1.0
    return image_batch

In [ ]:
def classify_local(image, model):
    if not model.is_local:
        print("model is remote try .get()")
        return -1, torch.Tensor([-1])
    image_tensor = torch.Tensor(prep_for_inference(image))
    output = model(image_tensor)
    preds = torch.exp(output)
    local_y = preds
    local_y = local_y.squeeze()
    pos = local_y == max(local_y)
    index = torch.nonzero(pos, as_tuple=False)
    class_num = index.squeeze()
    return class_num, local_y

In [ ]:
def classify_remote(image, model):
    if model.is_local:
        print("model is local try .send()")
        return -1, remote_torch.Tensor([-1])
    image_tensor_ptr = remote_torch.Tensor(prep_for_inference(image))
    output = model(image_tensor_ptr)
    preds = remote_torch.exp(output)
    preds_result = preds.get(
        request_block=True,
        reason="To see a real world example of inference",
        timeout_secs=10
    )
    if preds_result is None:
        print("No permission to do inference, request again")
        return -1, torch.Tensor([-1])
    else:
        # now we have the local tensor we can use local torch
        local_y = torch.Tensor(preds_result)
        local_y = local_y.squeeze()
        pos = local_y == max(local_y)
        index = torch.nonzero(pos, as_tuple=False)
        class_num = index.squeeze()
        return class_num, local_y

In [ ]:
# lets grab something from the test set
import random
total_images = test_data_length # 10000
index = random.randint(0, total_images)
print("Random Test Image:", index)
count = 0
batch = index // test_kwargs["batch_size"]
batch_index = index % int(total_images / len(test_loader))
for tensor_ptr in test_loader:
    data, target = tensor_ptr[0], tensor_ptr[1]
    if batch == count:
        break
    count += 1

print(f"Displaying {index} == {batch_index} in Batch: {batch}/{len(test_loader)}")
if batch_index > len(data):
    batch_index = 0
image_1 = data[batch_index].reshape((28, 28))
label_1 = target[batch_index]
draw_image_and_label(image_1, label_1)

In [ ]:
# classify remote
class_num, preds = classify_remote(image_1, model)
print(f"Prediction: {class_num} Ground Truth: {label_1}")
print(preds)

In [ ]:
if local_model is None:
    local_model = model.get(
        request_block=True,
        reason="To run test and inference locally",
        timeout_secs=5,
    )

In [ ]:
# classify local
class_num, preds = classify_local(image_1, local_model)
print(f"Prediction: {class_num} Ground Truth: {label_1}")
print(preds)

In [ ]:
# We can also download an image from the web and run inference on that
from PIL import Image, ImageEnhance
import PIL.ImageOps    

import os
def classify_url_image(image_url):
    filename = os.path.basename(image_url)
    os.system(f'curl -O {image_url}')
    im = Image.open(filename)
    im = PIL.ImageOps.invert(im)
#     im = im.resize((28,28), Image.ANTIALIAS)
    im = im.convert('LA')
    enhancer = ImageEnhance.Brightness(im)
    im = enhancer.enhance(3)


    print(im.size)
    fig = plt.figure()
    plt.tight_layout()
    plt.imshow(im, cmap="gray", interpolation="none")
    
    # classify local
    class_num, preds = classify_local(image_1, local_model)
    print(f"Prediction: {class_num}")
    print(preds)

In [ ]:
# image_url = "https://raw.githubusercontent.com/kensanata/numbers/master/0018_CHXX/0/number-100.png"
# classify_url_image(image_url)

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Owner notebook until the next checkpoint.